In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# imdb_reviews = pd.read_csv('https://raw.githubusercontent.com/naserahmadi/TDmatch/main/data/imdb/IMDB_reviews.csv', names = ['text'])
# imdb_reviews['row_number'] = imdb_reviews.reset_index().index
# imdb_reviews.to_csv('data/imdb_raw_reviews.csv')

imdb_raw_reviews = pd.read_csv('data/imdb_raw_reviews.csv')
imdb_raw_reviews.head()

,text
0,the shawshank redemption is without a doubt on...
1,the shawshank redemption is written and direct...
2,the godfather is one of the very few films tha...
3,'the godfather' is the pinnacle of flawless fi...
4,dark yes complex ambitious. christopher nol...


In [3]:
imdb_raw_reviews.shape

(500, 1)

In [4]:
imdb_reviews = pd.read_csv('data/imdb_reviews.csv')[['movie', 'user_review_permalink', 'user_review']]
imdb_reviews.head()

,movie,user_review_permalink,user_review
0,The Shawshank Redemption,https://www.imdb.com/review/rw3870888/,i watch this movie for minimum 5-6 times first...
1,The Shawshank Redemption,https://www.imdb.com/review/rw2284594/,The Shawshank Redemption is written and direct...
2,The Godfather,https://www.imdb.com/review/rw3205452/,The Godfather is one of the most iconic films ...
3,The Godfather,https://www.imdb.com/review/rw3038370/,'The Godfather' is the pinnacle of flawless fi...
4,The Dark Knight,https://www.imdb.com/review/rw1917099/,"Dark, yes, complex, ambitious. Christopher Nol..."


In [5]:
imdb_reviews.shape

(2000, 3)

# ChatGPT Entity Extraction Testing

In [12]:
# https://github.com/openai/openai-python
import yaml
import openai

with open("openai.yaml", "r") as stream:
    d = yaml.safe_load(stream)

openai.api_key = d['openai']['api-key']

In [17]:
help(chatgpt.create)

Help on method create in module openai.api_resources.chat_completion:

create(*args, **kwargs) method of builtins.type instance
    Creates a new chat completion for the provided messages and parameters.
    
    See https://platform.openai.com/docs/api-reference/chat-completions/create
    for a list of valid parameters.



In [18]:
model = 'gpt-3.5-turbo'
prompt = 'Hi'

chatgpt = openai.ChatCompletion()

completion = chatgpt.create(model=model, messages=[{"role": "user", "content": prompt}], temperature=0)
print(completion.choices[0].message.content)

Hello! How can I assist you today?


## Extracting a specific entity from text

In [19]:
import time

# extracts one entity from text 
def get_entity(entity, text):
    
    # chill for a second
    # time.sleep(1)
    
    wait = True
    seconds = 1
    retries = 1
    
    messages = [
        {"role": "user", "content": f"Your job is to extract the {entity} being described by the text provided."},
        {"role": "user", "content": text},
        {"role": "user", "content": f"Respond ONLY with the {entity} ONLY {entity}"},
        {"role": "user", "content": f"If {entity} is not found, respond ONLY with Unknown ONLY Unknown"}
    ]

    while wait:
        try:
            response = chatgpt.create(model=model, messages=messages, temperature=0).choices[0].message.content
        except:
            time.sleep(seconds)
            print(f'Retry #{retries}, waiting for an additional {seconds} seconds...')
            retries += 1
            seconds += 1
            
            if retries > 20:
                return 'Unknown'
        else:
            return response

In [20]:
test_review = '''i'm not an al pacino fan but this movie has the best performance of al. i think no one in the history of world cinema has played a character better than al pacino in godfather 2. he deserved an oscar.
godfather 2 is better than the first part. it is a great sequel  it expands the themes and the world of the first part. robert de niro is stunning as usual. missed marlon brando. best part of this movie is it shows the rise of vito corleone and michael corleone contrasting their times and personalities.
you need to watch godfather 2'''

In [22]:
entity_to_extract = "official title of the movie"
get_entity(entity_to_extract, test_review)

'The Godfather: Part II'

In [23]:
entity_to_extract = "lead actor of the movie"
get_entity(entity_to_extract, test_review)

'Al Pacino'

In [24]:
entity_to_extract = "all entities, and the type of entity"
get_entity(entity_to_extract, test_review)

'Entities:\n- Al Pacino (Person)\n- Godfather 2 (Movie)\n- World Cinema (Concept)\n- Character (Concept)\n- Oscar (Award)\n- First Part (Movie)\n- Sequel (Concept)\n- Themes (Concept)\n- Robert De Niro (Person)\n- Marlon Brando (Person)\n- Vito Corleone (Character)\n- Michael Corleone (Character)\n\nNote: Unknown entities are not found in the text.'

# IMDB Uncertain Reviews After Flair

In [192]:
imdb_uncertain = pd.read_csv('data/imdb_uncertain_after_flair.csv')['id'].sort_values().unique().tolist()

In [193]:
# filter imdb_reviews based on ids in the imdb_uncertain dataframe
ids_filter = imdb_reviews.index.isin(imdb_uncertain)
imdb_reviews_uncertain = imdb_reviews.copy()[ids_filter].reset_index(drop=True)
imdb_reviews_uncertain.head()

,movie,user_review_permalink,user_review
0,The Shawshank Redemption,https://www.imdb.com/review/rw3870888/,i watch this movie for minimum 5-6 times first...
1,The Dark Knight,https://www.imdb.com/review/rw1917099/,"Dark, yes, complex, ambitious. Christopher Nol..."
2,The Dark Knight,https://www.imdb.com/review/rw5478826/,"Confidently directed, dark, brooding, and pack..."
3,The Godfather: Part II,https://www.imdb.com/review/rw0135607/,"Coppola's masterpiece is rivaled only by ""The ..."
4,12 Angry Men,https://www.imdb.com/review/rw0059999/,An excellent courtroom drama with a unique twi...


## Test on first 5 columns

In [182]:
test = imdb_reviews.copy().head()

In [183]:
test

,movie,user_review_permalink,user_review
0,The Shawshank Redemption,https://www.imdb.com/review/rw3870888/,i watch this movie for minimum 5-6 times first...
1,The Shawshank Redemption,https://www.imdb.com/review/rw2284594/,The Shawshank Redemption is written and direct...
2,The Godfather,https://www.imdb.com/review/rw3205452/,The Godfather is one of the most iconic films ...
3,The Godfather,https://www.imdb.com/review/rw3038370/,'The Godfather' is the pinnacle of flawless fi...
4,The Dark Knight,https://www.imdb.com/review/rw1917099/,"Dark, yes, complex, ambitious. Christopher Nol..."


In [184]:
total = len(test)-1

def get_entity_wrapper(row):
    print(f'Running row {row.name} / {total}')
    return get_entity("official title of the movie", row['user_review'])

In [185]:
total

4

In [186]:
test['gpt_movie_title'] = test.apply(lambda row: get_entity_wrapper(row), axis=1)

Running row 0 / 4
Running row 1 / 4
Running row 2 / 4
Running row 3 / 4
Running row 4 / 4


In [187]:
test

,movie,user_review_permalink,user_review,gpt_movie_title
0,The Shawshank Redemption,https://www.imdb.com/review/rw3870888/,i watch this movie for minimum 5-6 times first...,Unknown.
1,The Shawshank Redemption,https://www.imdb.com/review/rw2284594/,The Shawshank Redemption is written and direct...,The Shawshank Redemption
2,The Godfather,https://www.imdb.com/review/rw3205452/,The Godfather is one of the most iconic films ...,The Godfather.
3,The Godfather,https://www.imdb.com/review/rw3038370/,'The Godfather' is the pinnacle of flawless fi...,The Godfather.
4,The Dark Knight,https://www.imdb.com/review/rw1917099/,"Dark, yes, complex, ambitious. Christopher Nol...",The Dark Knight.


## Run on entire imdb dataframe

In [194]:
total = len(imdb_reviews_uncertain)-1

def get_entity_wrapper(row):
    print(f'Running row {row.name} / {total}')
    return get_entity("official title of the movie", row['user_review'])

In [195]:
total

1381

In [ ]:
imdb_reviews_uncertain['gpt_movie_title'] = imdb_reviews_uncertain.apply(lambda row: get_entity_wrapper(row), axis=1)

In [199]:
imdb_reviews_uncertain.to_csv('data/imdb_after_gpt.csv')

## GPT Post Processing

In [214]:
imdb_reviews_uncertain = pd.read_csv('data/imdb_after_gpt.csv').iloc[:, 1:]

In [215]:
imdb_reviews_uncertain.head()

,movie,user_review_permalink,user_review,gpt_movie_title
0,The Shawshank Redemption,https://www.imdb.com/review/rw3870888/,i watch this movie for minimum 5-6 times first...,Unknown.
1,The Dark Knight,https://www.imdb.com/review/rw1917099/,"Dark, yes, complex, ambitious. Christopher Nol...",The Dark Knight
2,The Dark Knight,https://www.imdb.com/review/rw5478826/,"Confidently directed, dark, brooding, and pack...",The Dark Knight
3,The Godfather: Part II,https://www.imdb.com/review/rw0135607/,"Coppola's masterpiece is rivaled only by ""The ...","The Godfather, Part II."
4,12 Angry Men,https://www.imdb.com/review/rw0059999/,An excellent courtroom drama with a unique twi...,12 Angry Men


In [216]:
def remove_ending_period(string):
    if string[-1] == '.':
        return string[:-1]
    else:
        return string

In [217]:
imdb_reviews_uncertain['gpt_movie_title'] = imdb_reviews_uncertain['gpt_movie_title'].apply(remove_ending_period)
imdb_reviews_uncertain['gpt_movie_title'] = imdb_reviews_uncertain['gpt_movie_title'].replace('Unknown', 'N/A')

In [234]:
filt = imdb_reviews_uncertain['gpt_movie_title'].notna()
imdb_reviews_uncertain[filt][['user_review', 'gpt_movie_title']].head(15)

,user_review,gpt_movie_title
1,"Dark, yes, complex, ambitious. Christopher Nol...",The Dark Knight
2,"Confidently directed, dark, brooding, and pack...",The Dark Knight
3,"Coppola's masterpiece is rivaled only by ""The ...","The Godfather, Part II"
4,An excellent courtroom drama with a unique twi...,12 Angry Men
5,I've just re-watched The Lord of the Rings tri...,The Lord of the Rings: The Return of the King
6,This Movie was sensational. It was a piece of ...,Schindler's List
7,"Is it just me, or was this unintelligible? Not...",Inception
8,Fight Club is one of those movies that hits th...,Fight Club
9,Simply incredible. Never before have I seen a ...,The Lord of the Rings: The Fellowship of the Ring
10,Simply incredible. Never before have I seen a ...,The Lord of the Rings: The Fellowship of the Ring


In [219]:
imdb_reviews_uncertain.to_csv('data/imdb_after_gpt.csv')

In [220]:
imdb_reviews_uncertain = pd.read_csv('data/imdb_after_gpt.csv').iloc[:, 1:]

# GPT Movie Title Analysis

In [227]:
round(np.sum(imdb_reviews_uncertain['movie'] == imdb_reviews_uncertain['gpt_movie_title'])/len(imdb_reviews_uncertain), 2)

0.53

### There are a lot of instances where gpt got it correct, but the punctuation or capitaliztion is slightly off - needs additional processing to evaluate

# COVID Dataset

In [47]:
covid_df = pd.read_csv('data/covid_raw_user_sentences_w_ids.csv').iloc[:, 1:]

In [48]:
covid_df.shape

(50, 6)

In [49]:
covid_df.head()

,id,sentence,country_1,country_2,count_type1,count_type2
0,0,united states has 3 times the cases of canada,united states,canada,total confirmed,total confirmed
1,1,the number of cases in france is higher than i...,france,germany,total confirmed,total confirmed
2,2,the total death in russia is below 10000.,russia,NaN,total deaths,NaN
3,3,france has the most confirmed cases in europe.,france,NaN,total confirmed,NaN
4,4,china and japan have a total of over 5 million...,china,japan,total confirmed,total confirmed


In [50]:
categories = ['total deaths',
              'new deaths',
              'total recovered',
              'new recovered',
              'total convirmed',
              'new confirmed']

In [51]:
# extracts one category from a list of categories that the text describes
def get_category(categories, text):
    
    # chill for a second
    # time.sleep(1)
    
    wait = True
    seconds = 1
    retries = 1
    
    messages = [
        {"role": "user", "content": f"Your job is to extract the category being described by the text provided."},
        {"role": "user", "content": f"You can only choose from the following categories: {categories}"},
        {"role": "user", "content": text},
        {"role": "user", "content": f"Respond ONLY with the category ONLY category"},
        {"role": "user", "content": f"If a category is not found, respond ONLY with Unknown ONLY Unknown"}
    ]

    while wait:
        try:
            response = chatgpt.create(model=model, messages=messages, temperature=0).choices[0].message.content
        except:
            time.sleep(seconds)
            print(f'Retry #{retries}, waiting for an additional {seconds} seconds...')
            retries += 1
            seconds += 1
            
            if retries > 20:
                return 'Unknown'
        else:
            return response

In [52]:
get_category(categories, 'the total death in russia is below 10000.')

'total deaths'

## Run on entire COVID dataframe

In [53]:
total = len(covid_df)-1

def get_category_wrapper(row):
    print(f'Running row {row.name} / {total}')
    return get_category(categories, row['sentence'])

In [54]:
total

49

In [55]:
covid_df['count_type_gpt'] = covid_df.apply(lambda row: get_category_wrapper(row), axis=1)

Running row 0 / 49
Running row 1 / 49
Running row 2 / 49
Running row 3 / 49
Running row 4 / 49
Running row 5 / 49
Running row 6 / 49
Running row 7 / 49
Running row 8 / 49
Running row 9 / 49
Running row 10 / 49
Running row 11 / 49
Running row 12 / 49
Running row 13 / 49
Running row 14 / 49
Running row 15 / 49
Running row 16 / 49
Running row 17 / 49
Running row 18 / 49
Running row 19 / 49
Running row 20 / 49
Running row 21 / 49
Running row 22 / 49
Running row 23 / 49
Running row 24 / 49
Running row 25 / 49
Running row 26 / 49
Running row 27 / 49
Running row 28 / 49
Running row 29 / 49
Running row 30 / 49
Running row 31 / 49
Running row 32 / 49
Running row 33 / 49
Running row 34 / 49
Running row 35 / 49
Running row 36 / 49
Running row 37 / 49
Running row 38 / 49
Running row 39 / 49
Running row 40 / 49
Running row 41 / 49
Running row 42 / 49
Running row 43 / 49
Running row 44 / 49
Running row 45 / 49
Running row 46 / 49
Running row 47 / 49
Running row 48 / 49
Running row 49 / 49


In [56]:
covid_df.to_csv('data/covid_raw_user_sentences_w_ids_after_gpt.csv')

## GPT Post Processing

In [59]:
covid_df = pd.read_csv('data/covid_raw_user_sentences_w_ids_after_gpt.csv').iloc[:, 1:]

In [61]:
covid_df.head()

,id,sentence,country_1,country_2,count_type1,count_type2,count_type_gpt
0,0,united states has 3 times the cases of canada,united states,canada,total confirmed,total confirmed,new confirmed
1,1,the number of cases in france is higher than i...,france,germany,total confirmed,total confirmed,new confirmed
2,2,the total death in russia is below 10000.,russia,NaN,total deaths,NaN,total deaths
3,3,france has the most confirmed cases in europe.,france,NaN,total confirmed,NaN,total confirmed
4,4,china and japan have a total of over 5 million...,china,japan,total confirmed,total confirmed,total confirmed
